# Interacting with data through the Cubeviz application

We first start the the application in the Cubeviz configuration.

In [ ]:
# Suppress warnings
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [ ]:
from astropy.utils.data import download_file

from jdaviz.app import Application
app = Application(configuration='cubeviz')
app

Next, we load an IFU cube data set. **Note** this does not yet auto-populate viewers, still investigating heuristics on how to associate particular data sets with particular viewers.

In [ ]:
# This file is originally from https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/7495/stack/manga-7495-12704-LOGCUBE.fits.gz
# but has been modified to correct some inconsistencies in the way units are parsed
fn = download_file('https://stsci.box.com/shared/static/28a88k1qfipo4yxc4p4d40v4axtlal8y.fits', cache=True)
app.load_data(fn)

Before we can continue on, we manually set the data within each viewer. After having done that, we can gain access to the viewer of interest by the reference defined in the Cubeviz `yaml` configuration file. In this case, we've defined each viewer as follows:


```
...
viewer_area:
  - container: col
    children:
      - container: row
        viewers:
          - name: Flux
            plot: g-image-viewer
            reference: flux-viewer
          - name: Uncertainty
            plot: g-image-viewer
            reference: uncert-viewer
          - name: Mask
            plot: g-image-viewer
            reference: mask-viewer
      - container: row
        viewers:
          - name: Spectrum
            plot: g-profile-viewer
            reference: spectrum-viewer
```

Note the `reference` keyword that associated the viewer with a user-readable string name.

Then, we can access the individual viewer objects through the `get_viewer` method on the `app` instance:

<div class="alert alert-block alert-info">
<b>User Task:</b> In the top left viewer in the Cubeviz application (our `flux-viewer`), open the "Data" tab and plot the "6de4c8ee5659e87a302e3de595074ba5[FLUX]" data set by clicking the checkbox.
</div>

In [ ]:
app.get_viewer('spectrum-viewer').show()

We now load the data into the spectrum viewer manually using the current API by calling `set_viewer_data` and passing the viewer reference and data label.

In [ ]:
app.add_data_to_viewer('spectrum-viewer', '6de4c8ee5659e87a302e3de595074ba5[FLUX]')

To retrieve particular data from a viewer, we use the `get_data` method, passing in the reference to the viewer of interest. Optionally, we can pass in a class that will be used in the conversion from the `glue` data objects. If no `cls` argument is set, it will fall back to the `default_class` attribute set on the viewer. In the case of the `cubeviz-spectrum-viewer`, this is a `Spectrum1D` object.

Note that this returns all data currently plotted in the viewer.

In [ ]:
from specutils import Spectrum1D

spec_data = app.get_viewer('spectrum-viewer').data()

# The returned data from `get_data` is in list format, as it's 
# possible for there to be several data plotted in the viewer
spec = spec_data[0]
spec

Optionally, we can specify the label explicitly in the `get_viewer_data` call to retrieve a specific layer.

In [ ]:
spec = app._jdaviz_helper.get_data('6de4c8ee5659e87a302e3de595074ba5[FLUX]', function='sum')
spec

We then import some analysis or manipulation function we want to perform over the extracted spectral data in the previous cell.

In [ ]:
from specutils.manipulation import box_smooth

smoothed_spec = box_smooth(spec, 5)

Once we have the results from our manipulation/analysis function, we can insert the data into the appropriate viewer using the `add_data` method on the `app` object and passing in the requisite viewer reference.

In [ ]:
# This is necessary since the wcs is not being auto-linked in the manga data. If
#  we don't unset the previous data set, the new data won't be shown on the plot.
app.remove_data_from_viewer('spectrum-viewer', '6de4c8ee5659e87a302e3de595074ba5[FLUX]')

In [ ]:
app.add_data(smoothed_spec, data_label="Smooth Spectrum")

In [ ]:
app.add_data_to_viewer('spectrum-viewer', 'Smooth Spectrum')

## Retrieving manipulated data products

Here, we'll go back up to the Cubeviz application and use the collapse plugin to create a 2D data product from within the application. Afterward, we plot it in the `flux-viewer` viewer and then we can retrieve and view the data below in both the glue-jupyter image viewer as well as retrieving and the data and showing it in a matplotlib plot.

<div class="alert alert-block alert-info">
<b>User Task:</b> In the Cubeviz application at the top, select the "Collapse" button in the toolbar and collapse the "6de4c8ee5659e87a302e3de595074ba5[FLUX]" data set along the 0th axis to create a 2D image.
    
Next, in the top-left viewer (our `flux-viewer`), uncheck any currently plotted data and scroll to the bottom to click the checkbox next to "Collapsed 6de4c8ee5659e87a302e3de595074ba5[FLUX]" to plot the 2D image.
</div>

In [ ]:
# Show the current flux viewer
app.get_viewer('flux-viewer').show()

In [ ]:
import matplotlib.pyplot as plt

# The returned data object is a `CCData` class to represent the 2D nature of the image data.
image_data = app.get_viewer('flux-viewer').data()[0]

f, ax = plt.subplots()

ax.imshow(image_data[0], origin='center')